In [2]:
# Imports
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
import time
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import feature_extraction as fe
from aicsshparam import shtools, shparam # spherical harmonics package
import seaborn as sns
import sparse
import align_3d as align

import json
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
import argparse
import time
import sys
import os

import numpy as np
import pandas as pd
import sparse
import dask.array as da
from aicsshparam import shtools, shparam
from dask import delayed, compute

from utils.dask_utils import setup_dask_sge_cluster, shutdown_dask
from dask.distributed import Client, LocalCluster, get_client

mask_path = '/net/beliveau/vol2/instrument/E9.5_290/Zoom_290_subset_test/dataset_fused_masks_cpsamr5.zarr'
n5_image_path = '/net/beliveau/vol2/instrument/E9.5_290/Zoom_290_subset_test/dataset_fused.n5'

# def load_all_data(mask_path, 
#                   n5_path, 
#                   crop_size=None,
#                   subset_size=None):
#     # load mask and image data
#     mask_da = fe.load_n5_zarr_array(mask_path)
#     image_da = fe.load_n5_zarr_array(n5_path, n5_subpath='ch0/s0')

#     if crop_size is not None:
#         mask_da = mask_da[:crop_size[0], :crop_size[1], :crop_size[2]]
#         image_da = image_da[:crop_size[0], :crop_size[1], :crop_size[2]]

#     # convert to sparse
#     chunk_shape = tuple(c[0] for c in mask_da.chunks)
#     meta_block = sparse.COO.from_numpy(np.zeros(chunk_shape, 
#                                                 dtype=mask_da.dtype))
#     mask_sparse = mask_da.map_blocks(
#     fe.to_sparse,
#     dtype=mask_da.dtype,
#     meta=meta_block,
#     chunks=mask_da.chunks
#     )

#     # find bounding boxes
#     df_bboxes = fe.find_objects(mask_sparse).compute()
#     df_bboxes = pd.DataFrame(df_bboxes)
#     print(f"Found {len(df_bboxes)} objects")

#     if subset_size is not None:
#         obj_idxs = np.random.randint(0, 
#                                      len(df_bboxes), 
#                                      size=subset_size)
#         test_objects = df_bboxes.iloc[obj_idxs]
#         print(df_bboxes.head(3))
#         return mask_da, image_da, test_objects
#     else: 
#         print(df_bboxes.head(3))
#         return mask_da, image_da, df_bboxes
# mask_da, image_da, df_bboxes = load_all_data(mask_path, n5_image_path, crop_size=(800, 800, 800), subset_size=5)

2025-07-07 10:51:46,434 - INFO - Successfuly imported local module
2025-07-07 10:51:46,436 - INFO - --- Environment Versions ---
2025-07-07 10:51:46,438 - INFO - Platform: Linux-5.15.0-119-generic-x86_64-with-glibc2.35
2025-07-07 10:51:46,439 - INFO - Python: 3.13.3 | packaged by conda-forge | (main, Apr 14 2025, 20:44:03) [GCC 13.3.0]
2025-07-07 10:51:46,440 - INFO - Dask: 2025.2.0
2025-07-07 10:51:46,443 - INFO - Distributed: 2025.2.0
2025-07-07 10:51:46,444 - INFO - Cloudpickle: 3.0.0
2025-07-07 10:51:46,445 - INFO - Msgpack: 1.0.8
2025-07-07 10:51:46,447 - INFO - Zarr: 2.13.3
2025-07-07 10:51:46,448 - INFO - NumPy: 2.2.6
2025-07-07 10:51:46,450 - INFO - Scikit-image: 0.25.0
2025-07-07 10:51:46,451 - INFO - --- Dask Config (relevant parts) ---
2025-07-07 10:51:46,452 - INFO - distributed.comm.compression: False
2025-07-07 10:51:46,453 - INFO - --- End Environment Info ---


In [1]:
import logging
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from dask import delayed, compute
import sparse
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

# Import your modules
import align_3d as align
import feature_extraction as fe
from aicsshparam import shtools, shparam
from utils.dask_utils import setup_dask_sge_cluster, shutdown_dask

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

DEFAULT_LMAX = 12
MIN_RELIABLE_OBJECTS = 30

#TODO: add a n > p check

def process_individual(obj, mask_da, min_size):
    """Process a single object in distributed manner."""
    try:
        slice_z, slice_y, slice_x = obj[0], obj[1], obj[2]
        obj_id = int(obj.name)
        
        mask_slice = mask_da[slice_z, slice_y, slice_x].compute()
        label_slice = np.where(mask_slice == obj_id, mask_slice, 0)
        
        if np.sum(label_slice > 0) < min_size:
            logger.debug(f"Object {obj_id} is too small")
            return None
            
        aligned_obj, _ = align.align_object(label_slice, {})
        return {'obj_id': obj_id, 'aligned_obj': aligned_obj}
    except Exception as e:
        logger.debug(f"Failed to process object {obj.name}: {e}")
        return None


def get_coeffs_and_errors(object_batch, lmax_range):
    """Compute both reconstruction errors and coefficients for a batch of objects."""
    results = {}
    
    for obj_data in object_batch:
        if obj_data is None:
            continue
            
        obj_id = obj_data['obj_id']
        aligned_obj = obj_data['aligned_obj']
        obj_results = {'errors': {}, 'coeffs': {}}
        
        for lmax in lmax_range:
            try:
                # Get coefficients and reconstruction in one call
                (coeffs_dict, grid_rec), (_, _, grid, _) = shparam.get_shcoeffs(
                    aligned_obj, lmax=lmax, alignment_2d=False
                )
                
                # Compute reconstruction error
                mse = shtools.get_reconstruction_error(grid, grid_rec)
                
                if mse is not None and not np.isnan(mse):
                    obj_results['errors'][lmax] = mse
                    obj_results['coeffs'][lmax] = coeffs_dict
                    
            except Exception as e:
                logger.debug(f"Failed computation for object {obj_id} lmax {lmax}: {e}")
        
        if obj_results['errors']:  # If we got at least one valid result
            results[obj_id] = obj_results

    return results


class SphericalHarmonicsLOOCV:
    """
    Leave-One-Out Cross-Validation for selecting optimal lmax for spherical harmonics
    decomposition of 3D objects from a mouse brain sample.
    """
    
    def __init__(self, params):
        self.params = params
        self.client = None
        self.cluster = None
        self.lmax_range = list(range(self.params['lmax_min'], 
                                     self.params['lmax_max'], 
                                     4))
        
    def select_optimal_lmax(self, mask_path):
        """
        Main entry point: finds optimal lmax for this mouse sample.
        
        Args:
            mask_path: Path to segmentation mask
            n5_path: Path to n5 image data (optional, not used currently)
        
        Returns:
            optimal_lmax: Best lmax value
            cv_results: Cross-validation results for each lmax
        """
        try:
            # Setup distributed computing
            if self.params.get('use_dask', True):
                self._setup_dask()
            
            # Load and process objects
            mask_da, df_bboxes = self._load_data(mask_path)
            processed_objects = self._process_objects_batch(mask_da, df_bboxes)
            
            if len(processed_objects) < MIN_RELIABLE_OBJECTS:
                logger.warning(f"Only {len(processed_objects)} valid objects found. Results may be unreliable.")
            
            # Compute coefficients and errors for all lmax values
            all_data = self._precompute_all(processed_objects)
            
            # Run LOOCV evaluation
            cv_results = self._evaluate_lmax_loocv(all_data)
            
            # Find optimal lmax
            optimal_lmax = self._find_optimal_lmax(cv_results)
            
            return optimal_lmax, cv_results
            
        finally:
            # Always cleanup
            if self.client:
                self._shutdown_dask()
    
    def _load_data(self, mask_path):
        """Load objects from mask."""
        logger.info("Loading data and finding objects...")
        mask_da = fe.load_n5_zarr_array(mask_path)
        
        # Find objects using sparse representation
        chunk_shape = tuple(c[0] for c in mask_da.chunks)
        meta_block = sparse.COO.from_numpy(np.zeros(chunk_shape, dtype=mask_da.dtype))
        mask_sparse = mask_da.map_blocks(
            fe.to_sparse, 
            dtype=mask_da.dtype, 
            meta=meta_block, 
            chunks=mask_da.chunks
        )
        df_bboxes = fe.find_objects(mask_sparse).compute()
        df_bboxes = pd.DataFrame(df_bboxes)
        logger.info(f"Found {len(df_bboxes)} total objects")
        return mask_da, df_bboxes
    
    def _process_objects_vanilla(self, mask_da, df_bboxes):
        n_sample = min(self.params['sample_size'], len(df_bboxes))
        sample = df_bboxes.sample(n_sample)
        logger.info(f"Sampled {len(sample)} objects")

        all_objects = []
        for _, obj in sample.iterrows():
            obj_data = process_individual(obj, mask_da, self.params['min_object_size'])
            if obj_data is not None:
                all_objects.append(obj_data)
            else: 
                logger.info("failed to process object")
        if len(all_objects) == 0:
            logger.debug("No valid objects found")
            return None
        
        logger.info(f"Successfully processed {len(all_objects)} valid objects")
        return all_objects
    
    
    def _process_objects_batch(self, mask_da, df_bboxes):
        """Sample and create delayed processing tasks for objects."""
        logger.info("Creating delayed tasks for object pre-processing...")
        
        n_sample = min(self.params['sample_size'], len(df_bboxes))
        sample = df_bboxes.sample(n_sample, random_state=42)  # Add random_state for reproducibility
        logger.info(f"Sampled {len(sample)} objects")
        
        # Create delayed tasks for object processing
        delayed_tasks = []
        for _, obj in sample.iterrows():
            delayed_task = delayed(process_individual)(
                obj, mask_da, self.params['min_object_size']
            )
            delayed_tasks.append(delayed_task)
        
        logger.info(f"Created {len(delayed_tasks)} delayed object processing tasks.")
        return delayed_tasks
    
    def _precompute_all(self, processed_objects):
        """Compute coefficients and errors for all objects and lmax values."""
        lmax_range = range(self.params['lmax_min'], self.params['lmax_max'] + 1)
        logger.info("Computing spherical harmonics data...")
        
        # Create delayed tasks for batches
        batch_size = self.params.get('error_batch_size', 20)
        delayed_tasks = []
        
        for i in range(0, len(processed_objects), batch_size):
            batch = processed_objects[i:i + batch_size]
            task = delayed(get_coeffs_and_errors)(batch, lmax_range)
            delayed_tasks.append(task)
        
        # Execute all tasks
        logger.info(f"Computing data for {len(delayed_tasks)} batches...")
        batch_results = compute(*delayed_tasks)
        
        # Combine results
        all_data = {}
        for batch_result in batch_results:
            all_data.update(batch_result)
        
        logger.info(f"Computed data for {len(all_data)} objects")
        return all_data
    
    def _evaluate_lmax_loocv(self, all_data):
        """Evaluate each lmax using closed form LOOCV."""
        cv_results = {}
        for lmax in self.lmax_range:
            logger.info(f"Evaluating lmax={lmax}")
            
            # Extract residuals and coefficients for this lmax
            residuals = []
            coeffs_dicts = []
            
            for obj_id, obj_data in all_data.items():
                if lmax in obj_data['errors'] and lmax in obj_data['coeffs']:
                    # Apply sampling rate to convert to physical units
                    error = obj_data['errors'][lmax] * self.params.get('sampling_rate', 2.752)
                    residuals.append(error)
                    coeffs_dicts.append(obj_data['coeffs'][lmax])
            
            # Build design matrix
            X = self._build_design_matrix(coeffs_dicts, lmax)
            
            # Compute LOOCV error
            residuals = np.array(residuals)
            cv_error = self._compute_loocv_error(residuals, X)
            
            cv_results[lmax] = {
                'cv_error': cv_error,
                'n_objects': len(residuals),
                'mean_residual': np.mean(residuals),
                'std_residual': np.std(residuals)
            }
            logger.info(f"  LOOCV error: {cv_error:.6f}, mean residual: {np.mean(residuals):.6f}")
        return cv_results
    
    def _get_coeff_mapping(self, lmax):
        """Create mapping from coefficient names to column indices"""""
        col_map = {}
        col_idx = 0
        for L in range(lmax + 1):
            for M in range(L + 1):
                if M == 0:
                    col_map[f"shcoeffs_L{L}M{M}C"] = col_idx
                    col_idx += 1
                else:
                    col_map[f"shcoeffs_L{L}M{M}C"] = col_idx
                    col_map[f"shcoeffs_L{L}M{M}S"] = col_idx + 1
                    col_idx += 2
        return col_map
    
    def _build_design_matrix(self, coeffs_dicts, lmax):
        """Build design matrix from coefficient dictionaries."""
        n = len(coeffs_dicts)
        p = (lmax + 1) ** 2
        if n <= p:
            raise ValueError(f"n <= p: {n} <= {p}. Insufficient data to compute design matrix. Please decrease lmax range or increase sample size.")
        
        # Create mapping from coefficient names to column indices
        col_map = self._get_coeff_mapping(lmax)

        # Build sparse matrix
        rows, cols, data = [], [], []
        for i, coeffs_dict in enumerate(coeffs_dicts):
            for key, value in coeffs_dict.items():
                if key in col_map and abs(value) > 1e-10:
                    rows.append(i)
                    cols.append(col_map[key])
                    data.append(value)
        
        # Create sparse matrix and convert to dense
        X_sparse = csr_matrix((data, (rows, cols)), shape=(n, p))
        return X_sparse
    
    def _compute_hat_diagonal(self, X_sparse, XtX_inv):
        n_samples = X_sparse.shape[0]
        hat_diagonal = np.zeros(n_samples)
        for i in range(n_samples):
            x_i = X_sparse.getrow(i).toarray().flatten()  # Get single row
            hat_diagonal[i] = x_i @ XtX_inv @ x_i  # Scalar computation
        return hat_diagonal
    
    def _compute_loocv_error(self, residuals, X_sparse):
        """Compute LOOCV error using closed-form solution."""
        try:
            # Hat matrix computation: H = X(X'X)^{-1}X'
            XtX = X_sparse.T @ X_sparse
            XtX_dense = XtX.toarray()
            # Compute (X.T @ X)^{-1}
            XtX_inv = np.linalg.pinv(XtX_dense) # Use pseudo-inverse for stability
            
            # Compute diagonal of hat matrix efficiently
            # H_diag = np.sum(X * (X @ XtX_inv), axis=1)
            H_diag = self._compute_hat_diagonal(X_sparse, XtX_inv)
            
            # Check numerical stability
            if np.any(H_diag >= 1.0 - 1e-10):
                logger.warning("Numerical instability detected in hat matrix")
                return float('inf')
            
            # LOOCV residuals
            loocv_residuals = residuals / (1 - H_diag)
            cv_error = np.mean(loocv_residuals ** 2)
            
            return cv_error
            
        except Exception as e:
            logger.error(f"Error in LOOCV computation: {e}")
            return float('inf')
    
    def _find_optimal_lmax(self, cv_results):
        """Find optimal lmax from CV results."""
        # Filter valid results
        valid_results = {
            lmax: res['cv_error'] 
            for lmax, res in cv_results.items() 
            if res['cv_error'] != float('inf')
        }
        
        if not valid_results:
            logger.warning(f"No valid CV results, using default lmax: {DEFAULT_LMAX}")
            return DEFAULT_LMAX
        
        # Find minimum error
        optimal_lmax = min(valid_results, key=valid_results.get)
        logger.info(f"Optimal lmax: {optimal_lmax} with CV error: {valid_results[optimal_lmax]:.6f}")
        return optimal_lmax
    
    def _setup_dask(self):
        """Setup Dask SGE cluster for distributed processing."""
        try:
            logger.info("Setting up distributed Dask cluster...")
            self.cluster, self.client = setup_dask_sge_cluster(
                n_workers=self.params.get('num_workers', 8),
                cores=self.params.get('cores_per_worker', 2),
                processes=self.params.get('processes', 1),
                memory=self.params.get('mem_per_worker', '30G'),
                project=self.params.get('project', 'beliveaulab'),
                queue=self.params.get('queue', 'beliveau-long.q'),
                runtime=self.params.get('runtime', '7200'),
                resource_spec=self.params.get('resource_spec', 'mfree=30G'),
                log_directory=self.params.get('log_dir', None),
                conda_env=self.params.get('conda_env', 'otls-pipeline')
            )
            logger.info(f"Dask dashboard link: {self.client.dashboard_link}")
            return True
        except Exception as e:
            logger.error(f"Failed to setup distributed cluster: {e}")
            return False

    def _shutdown_dask(self):
        """Shutdown Dask cluster."""
        if self.cluster and self.client:
            try:
                shutdown_dask(self.cluster, self.client)
                logger.info("Dask cluster shut down successfully")
            except Exception as e:
                logger.error(f"Error shutting down cluster: {e}")

2025-07-08 12:31:49,333 - INFO - Successfuly imported local module
2025-07-08 12:31:49,334 - INFO - --- Environment Versions ---
2025-07-08 12:31:49,335 - INFO - Platform: Linux-5.15.0-119-generic-x86_64-with-glibc2.35
2025-07-08 12:31:49,336 - INFO - Python: 3.13.3 | packaged by conda-forge | (main, Apr 14 2025, 20:44:03) [GCC 13.3.0]
2025-07-08 12:31:49,337 - INFO - Dask: 2025.2.0
2025-07-08 12:31:49,339 - INFO - Distributed: 2025.2.0
2025-07-08 12:31:49,340 - INFO - Cloudpickle: 3.0.0
2025-07-08 12:31:49,340 - INFO - Msgpack: 1.0.8
2025-07-08 12:31:49,341 - INFO - Zarr: 2.13.3
2025-07-08 12:31:49,341 - INFO - NumPy: 2.2.6
2025-07-08 12:31:49,343 - INFO - Scikit-image: 0.25.0
2025-07-08 12:31:49,343 - INFO - --- Dask Config (relevant parts) ---
2025-07-08 12:31:49,344 - INFO - distributed.comm.compression: False
2025-07-08 12:31:49,345 - INFO - --- End Environment Info ---


In [ ]:
params = {
    "num_workers": 1,
    "mem_per_worker": "64G",
    "cores_per_worker": 2,
    "sample_size": 100,
    "sampling_rate": 2.752,
    "min_object_size": 100,
    "lmax_min": 4,
    "lmax_max": 28,
    "default_lmax": 12,
    "error_batch_size": 25 # Process 25 objects at a time per dask task
}
mask_path = '/net/beliveau/vol2/instrument/E9.5_290/Zoom_290_subset_test/dataset_fused_masks_cpsamr5.zarr'
n5_image_path = '/net/beliveau/vol2/instrument/E9.5_290/Zoom_290_subset_test/dataset_fused.n5'

loocv_optimizer = SphericalHarmonicsLOOCV(params)
optimal_lmax, cv_results = loocv_optimizer.select_optimal_lmax(mask_path)
print(f"Optimal lmax: {optimal_lmax}")
print("CV results:", cv_results)

2025-07-08 12:31:49,359 - INFO - Setting up distributed Dask cluster...
2025-07-08 12:31:49,360 - INFO - Setting up Dask SGE Cluster...
2025-07-08 12:31:49,360 - INFO - Dask config applied: {'temporary-directory': '/tmp/17965644.1.beliveau-long.q', 'distributed.comm.timeouts.connect': '3600s', 'distributed.comm.timeouts.tcp': '3600s', 'distributed.worker.memory.spill': 0.7, 'distributed.worker.memory.pause': 0.9, 'distributed.worker.memory.terminate': 0.98, 'distributed.scheduler.work-stealing': True, 'distributed.scheduler.worker-saturation': 1.1}
2025-07-08 12:31:49,362 - INFO - Dask worker logs will be stored in: ./dask_worker_logs_20250708_123149
2025-07-08 12:31:49,362 - INFO - Using project: beliveaulab, queue: beliveau-long.q
2025-07-08 12:31:49,363 - INFO - Requesting 2 cores, 1 processes, 30G memory per worker.
2025-07-08 12:31:49,364 - INFO - Job extra directives: ['-P beliveaulab -pe serial 2', '-l mfree=30G']
2025-07-08 12:31:52,633 - INFO - Configured SGECluster. Requestin

KilledWorker: Attempted to run task 'finalize-124278f4-624f-4ba5-ae4f-ed7f346b8506' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://10.110.100.15:37967. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.